In [27]:
import pandas as pd
import re

In [ ]:
# Download wikipedia dump from https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles.xml.bz2
!wget https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles-multistream.xml.bz2

--2025-02-18 20:06:31--  https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles-multistream.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 2620:0:861:3:208:80:154:71, 208.80.154.71
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|2620:0:861:3:208:80:154:71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24642610651 (23G) [application/octet-stream]
Saving to: ‘enwiki-latest-pages-articles-multistream.xml.bz2’

enwiki-latest-pages 100%[===================>]  22.95G  4.59MB/s    in 86m 21s 

2025-02-18 21:32:52 (4.54 MB/s) - ‘enwiki-latest-pages-articles-multistream.xml.bz2’ saved [24642610651/24642610651]



In [ ]:
# Download index file from https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles-multistream-index.txt.bz2
# This file is used to find the byte offset of each article in the dump
!wget https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles-multistream-index.txt.bz2

--2025-02-18 20:00:23--  https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles-multistream-index.txt.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 2620:0:861:3:208:80:154:71, 208.80.154.71
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|2620:0:861:3:208:80:154:71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 267766383 (255M) [application/octet-stream]
Saving to: ‘enwiki-latest-pages-articles-multistream-index.txt.bz2’

enwiki-latest-pages 100%[===================>] 255.36M  4.51MB/s    in 56s     

2025-02-18 20:01:19 (4.54 MB/s) - ‘enwiki-latest-pages-articles-multistream-index.txt.bz2’ saved [267766383/267766383]



In [7]:
# Extract wikipedia index file
#!bzip2 -d enwiki-latest-pages-articles-multistream-index.txt.bz2

# Parse wikipedia index file to extract movie indices
%run -i ../tools/parse_movie_index.py --index_file enwiki-latest-pages-articles-multistream-index.txt --out_file movie_index.txt

In [ ]:
# Extract movie articles from wikipedia dump and create a csv file
# %run -i ../tools/process_wiki_dump.py --dump_file enwiki-latest-pages-articles-multistream.xml.bz2 --index_file movie_index.txt --out_file ../download/movies.csv

2025-02-18 22:36:20.248 | INFO     | __main__:main:128 - loading index...
2025-02-18 22:36:20.292 | INFO     | __main__:main:130 - index loaded.
2025-02-18 22:36:20.292 | INFO     | __main__:main:132 - open output CSV file...
2025-02-18 22:36:20.293 | INFO     | __main__:main:139 - out file opened
2025-02-18 22:36:20.294 | INFO     | __main__:main:141 - open wiki dump file...
2025-02-18 22:36:20.294 | INFO     | __main__:main:147 - wiki dump file opened.
2025-02-18 22:36:20.546 | INFO     | __main__:main:196 - Blue Velvet (film) added.
2025-02-18 22:36:20.747 | INFO     | __main__:main:196 - Buffy the Vampire Slayer (film) added.
2025-02-18 22:36:20.959 | INFO     | __main__:main:196 - Batman &amp; Robin (film) added.
2025-02-18 22:36:21.114 | INFO     | __main__:main:196 - Show Me Love (film) added.
2025-02-18 22:36:21.451 | INFO     | __main__:main:196 - Manufacturing Consent (film) added.
2025-02-18 22:36:21.667 | INFO     | __main__:main:196 - Miss Congeniality (film) added.
2025-0

In [77]:
# read movies database
movie_db = pd.read_csv('movies.csv').rename(columns=lambda x: x.strip().replace(" ", ""))


In [78]:
# print sample of movies database
movie_db.head()

,id,title,cast,plot,poster
0,3947,Blue Velvet (film),{{cast listing|<!-- Cast is in credits order ...,<!-- Per WP:FILMPLOT plot summaries for featu...,https://upload.wikimedia.org/wikipedia/en/f/fd...
1,4231,Buffy the Vampire Slayer (film),{{Cast listing|* [[Kristy Swanson]] as [[Buff...,[[Buffy Summers]] is a cheerleader at Hemery H...,https://upload.wikimedia.org/wikipedia/en/0/09...
2,4729,Batman &amp; Robin (film),{{div col}}* [[Arnold Schwarzenegger]] as [[V...,<!-- Per WP:FILMPLOT plot summaries for featu...,https://upload.wikimedia.org/wikipedia/en/3/37...
3,11585,Show Me Love (film),* [[Alexandra Dahlström]] as Elin Olsson* [[R...,Two girls Agnes and Elin attend school in th...,https://upload.wikimedia.org/wikipedia/en/9/96...
4,19055,Manufacturing Consent (film),None,The film presents and illustrates Chomsky and ...,https://upload.wikimedia.org/wikipedia/en/1/11...


In [79]:
# print information about movies database
movie_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16299 entries, 0 to 16298
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      16299 non-null  int64 
 1   title   16299 non-null  object
 2   cast    16299 non-null  object
 3   plot    16288 non-null  object
 4   poster  16299 non-null  object
dtypes: int64(1), object(4)
memory usage: 636.8+ KB


In [86]:
def clean_column(text):
    """Removes Wikipedia markup, unnecessary tags, and extracts names."""
    if pd.isna(text) or text.lower() == "none":
        return "Unknown"
    
    # Remove Wikipedia templates, comments, and unnecessary symbols
    text = re.sub(r"\{\{.*?\}\}", "", text)  # Remove Wikipedia template tags
    text = re.sub(r"<!--.*?-->", "", text)  # Remove HTML comments
    text = re.sub(r"\[\[(?:[^\]|]*\|)?([^\]]+)\]\]", r"\1", text)  # Extract names from Wikipedia links
    text = re.sub(r"\*", "", text)  # Remove asterisks used for listing
    
    return text.strip()


In [87]:
movie_db.dropna(inplace=True)

In [88]:
# Remove markup from cast and plot column
movie_db["cast"] = movie_db["cast"].apply(clean_column)
movie_db["plot"] = movie_db["plot"].apply(clean_column)


In [90]:
# Print clean data
movie_db

,id,title,cast,plot,poster
0,3947,Blue Velvet (film),,College student Jeffrey Beaumont returns to hi...,https://upload.wikimedia.org/wikipedia/en/f/fd...
1,4231,Buffy the Vampire Slayer (film),Appearing in uncredited roles are Ben Affleck ...,Buffy Summers is a cheerleader at Hemery High ...,https://upload.wikimedia.org/wikipedia/en/0/09...
2,4729,Batman &amp; Robin (film),Arnold Schwarzenegger as Dr. Victor Fries / Mr...,Batman and his partner Robin encounter a new...,https://upload.wikimedia.org/wikipedia/en/3/37...
3,11585,Show Me Love (film),Alexandra Dahlström as Elin Olsson Rebecka Lil...,Two girls Agnes and Elin attend school in th...,https://upload.wikimedia.org/wikipedia/en/9/96...
4,19055,Manufacturing Consent (film),Unknown,The film presents and illustrates Chomsky and ...,https://upload.wikimedia.org/wikipedia/en/1/11...
...,...,...,...,...,...
16293,79040652,Late Shift (film),Leonie Benesch as Floria Sonja Riesen as Bea S...,Floria a devoted surgical nurse in Switzerlan...,https://upload.wikimedia.org/wikipedia/en/e/e9...
16294,79045397,Ithaqua (film),Luke Hemsworth Kevin Durand Michael Pitt Craig...,Survivors in a remote outpost in 19th century ...,https://upload.wikimedia.org/wikipedia/en/d/df...
16295,79045789,DraftThe Bitter End (film),Joan Collins as Wallis Simpson,The film documents the later years in the life...,https://upload.wikimedia.org/wikipedia/en/d/df...
16296,79057912,Vari (film),Ahmed Sharumeel as Abeeru Ashiyath Zaheena as ...,Abeeru a dedicated yet emotionally distant me...,https://upload.wikimedia.org/wikipedia/en/0/07...


In [98]:
# print cast in sample row
print(movie_db["cast"].iloc[1])

Appearing in uncredited roles are Ben Affleck as a basketball player  Ricki Lake as Charlotte  Seth Green as a vampire  and Alexis Arquette as the vampire DJ.


In [115]:
def extract_actors(text):
    """Extract actors from the cast column."""
    if pd.isna(text) or text.lower() == "none" or text.lower() == "unknown":
        return ""
    actors = ", ".join(re.findall(r"\b([A-Z][a-z]+(?:\s[A-Z][a-z]+)*)\s+as", text))
    if actors == "":
        return text
    else:
        return actors

In [116]:
# Get the actors list
movie_db["actors"] = movie_db["cast"].apply(extract_actors)

In [117]:
movie_db

,id,title,cast,plot,poster,actors
0,3947,Blue Velvet (film),,College student Jeffrey Beaumont returns to hi...,https://upload.wikimedia.org/wikipedia/en/f/fd...,
1,4231,Buffy the Vampire Slayer (film),Appearing in uncredited roles are Ben Affleck ...,Buffy Summers is a cheerleader at Hemery High ...,https://upload.wikimedia.org/wikipedia/en/0/09...,"Ben Affleck, Ricki Lake, Seth Green, Alexis Ar..."
2,4729,Batman &amp; Robin (film),Arnold Schwarzenegger as Dr. Victor Fries / Mr...,Batman and his partner Robin encounter a new...,https://upload.wikimedia.org/wikipedia/en/3/37...,"Arnold Schwarzenegger, George Clooney, Eric Ll..."
3,11585,Show Me Love (film),Alexandra Dahlström as Elin Olsson Rebecka Lil...,Two girls Agnes and Elin attend school in th...,https://upload.wikimedia.org/wikipedia/en/9/96...,"Elin Olsson Rebecka Liljeberg, Agnes Ahlberg E..."
4,19055,Manufacturing Consent (film),Unknown,The film presents and illustrates Chomsky and ...,https://upload.wikimedia.org/wikipedia/en/1/11...,
...,...,...,...,...,...,...
16293,79040652,Late Shift (film),Leonie Benesch as Floria Sonja Riesen as Bea S...,Floria a devoted surgical nurse in Switzerlan...,https://upload.wikimedia.org/wikipedia/en/e/e9...,"Leonie Benesch, Floria Sonja Riesen, Bea Schmi..."
16294,79045397,Ithaqua (film),Luke Hemsworth Kevin Durand Michael Pitt Craig...,Survivors in a remote outpost in 19th century ...,https://upload.wikimedia.org/wikipedia/en/d/df...,Luke Hemsworth Kevin Durand Michael Pitt Craig...
16295,79045789,DraftThe Bitter End (film),Joan Collins as Wallis Simpson,The film documents the later years in the life...,https://upload.wikimedia.org/wikipedia/en/d/df...,Joan Collins
16296,79057912,Vari (film),Ahmed Sharumeel as Abeeru Ashiyath Zaheena as ...,Abeeru a dedicated yet emotionally distant me...,https://upload.wikimedia.org/wikipedia/en/0/07...,"Ahmed Sharumeel, Abeeru Ashiyath Zaheena, Shar..."
